In [ ]:
from config import *

In [26]:
# Model Settings
MODEL_NAME="all-MiniLM-L6-v2"
user_input = "What is Coelom"

########################
region = 'us-east-1' 
index_name = 'ncert'
service = 'es'
aos_host = aos_host

In [27]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
from requests_aws4auth import AWS4Auth
from sentence_transformers import SentenceTransformer
import pandas as pd
from azure_openai_helper import generate_answer_from_context

In [28]:
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region)


In [29]:
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

In [30]:
aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [31]:
model = SentenceTransformer(MODEL_NAME)

In [32]:
xq = model.encode([user_input],convert_to_tensor=True)
xq_list = xq.tolist()

In [33]:
xq_list[0]

[-0.042863715440034866, 0.041995562613010406, -0.056733760982751846, 0.027966050431132317, -0.03282560035586357, -0.053680047392845154, -0.053033631294965744, 0.02711592987179756, -0.0007371108513325453, 0.026679690927267075, 0.05748998746275902, -0.14638976752758026, -0.042849037796258926, 0.05368601158261299, -0.022275950759649277, -0.1332804411649704, 0.0108184814453125, -0.05613688379526138, 0.0021505674812942743, 0.008198272436857224, 0.03028068318963051, 0.038957685232162476, -0.019268125295639038, 0.025642411783337593, 0.044883161783218384, 0.03785828500986099, 0.025045769289135933, 0.030045848339796066, 0.046761654317379, -0.014629685319960117, -0.04331487789750099, -0.013703132048249245, 0.0323970727622509, 0.015361163765192032, -0.044113971292972565, 0.01904839463531971, 0.012730293907225132, -0.015088694170117378, 0.009063957259058952, -0.019090885296463966, -0.07781611382961273, 0.0628509670495987, 0.06096912547945976, -0.006804461590945721, -0.013890428468585014, 0.0261204

In [34]:
query={
    "size": 10,
    "query": {
        "knn": {
            "text_vector":{
                "vector":xq_list[0],
                "k":10
            }
        }
    }
}

res = aos_client.search(index=index_name, 
                       body=query,
                       stored_fields=["text"])

In [35]:
print("Got %d Hits:" % res['hits']['total']['value'])

Got 50 Hits:


In [36]:
query_result=[]
contexts =""

In [37]:
counter = 0

In [38]:
for hit in res['hits']['hits']:
        if counter > 3:
            break
        contexts +=  hit['fields']['text'][0]+"\n---\n"
        counter += 1

In [39]:
type(contexts)

<class 'str'>

In [40]:
for hit in res['hits']['hits']:
    row=[hit['_id'],hit['_score'],hit['fields']['text'][0]]
    query_result.append(row)

In [41]:
query_result_df = pd.DataFrame(data=query_result,columns=["_id","_score","text"])

In [42]:
display(query_result_df)

,_id,_score,text
0,4Cis6IkB6nyGYIwsLyZv,0.707898,1 (a) Radial symmetry\nRationalised 2023-24\n...
1,4Sis6IkB6nyGYIwsMCZr,0.612489,"In\nsome animals, the body cavity is not line..."
2,6iis6IkB6nyGYIwsOSYq,0.603072,6 Examples of Coelenterata indicating outline...
3,4iis6IkB6nyGYIwsMSZk,0.602643,3 Diagrammatic sectional view of :\n(a) Coelom...
4,-iis6IkB6nyGYIwsSCaA,0.565705,\nThis phylum consists of a small group of wor...
5,_Cis6IkB6nyGYIwsSiZl,0.564052,"These are bilaterally\nsymmetrical, triplobla..."
6,3iis6IkB6nyGYIwsLSZ-,0.558436,", any plane that passes through the centre\ndo..."
7,DCis6IkB6nyGYIwsXieT,0.555726,Endoskeleton is fully ossified (bony) and the...
8,Fyis6IkB6nyGYIwsaifZ,0.552549,Aschelminthes are\npseudocoelomates and incl...
9,HCis6IkB6nyGYIwsbyfb,0.550470,How useful is the study of the nature of body ...


In [43]:
len(contexts)

1666

In [44]:
from pprint import pprint
pprint(contexts)

('1 (a)  Radial symmetry\n'
 'Rationalised 2023-24\n'
 'ANIMAL KINGDOM 3939\n'
 '4.1.4 Coelom\n'
 'Presence or absence of a cavity between the body\n'
 'wall and the gut wall is very important in\n'
 'classification. The body cavity, which is lined\n'
 'by mesoderm is called coelom . Animals\n'
 'possessing coelom are called coelomates,  e.g.,\n'
 'annelids, molluscs, arthropods, echinoderms,\n'
 'hemichordates and chordates (Figure 4.3a)\n'
 '---\n'
 ' In\n'
 'some animals, the body cavity is not lined by\n'
 'mesoderm, instead, the mesoderm is present as\n'
 'scattered pouches in between the ectoderm and\n'
 'endoderm. Such a body cavity is called\n'
 'pseudocoelom and the animals possessing them\n'
 'are called pseudocoelomates,  e.g.,\n'
 'aschelminthes (Figure 4.3b). The animals in\n'
 'which the body cavity is absent are called\n'
 'acoelomates,  e.g., platyhelminthes (Figure 4.3c).Figure 4\n'
 '---\n'
 '6 Examples of  Coelenterata indicating outline of their body form :\n'
 '(a)

In [45]:
reply = generate_answer_from_context(user_input, contexts)

In [46]:
from pprint import pprint

In [47]:
pprint(reply)

('Coelom is a body cavity that is lined by mesoderm. It is present between the '
 'body wall and the gut wall in certain animals. Animals possessing a coelom '
 'are called coelomates, which include annelids, molluscs, arthropods, '
 'echinoderms, hemichordates, and chordates. The presence or absence of a '
 'coelom is an important factor in animal classification.')
